In [1]:
import itertools
import numpy as np

#Teneur en eau résiduelle
tr = [0.03]

#Teneur en eau à saturation
#ts=np.arange(0.36, 0.46, 0.02, 'float')
ts= [0.36]
#Teneur en eau initiale
#ti=np.arange(0.05, 0.13, 0.02, 'float')
ti = [0.07]
#Perméabilité à saturation
#Ks=np.arange(0.01, 0.56, 0.05, 'float')
Ks = [0.16000000000000003]
#param fitting retention n
#n=np.arange(1.5, 10.25, 0.25, 'float')
#n=np.arange(4, 4.75, 0.25, 'float')
n = [4]
#param fitting retention alpha
#alpha=np.arange(0.01, 0.11, 0.01, 'float')
#alpha=np.arange(0.01, 0.04, 0.01, 'float')
alpha = [0.01]

In [2]:
l= [ p for p in itertools.product(tr,ts,ti,Ks,n,alpha)]
len(l)

1

In [ ]:
import dask

In [3]:
import os

from modelisation import Geometry, ParamMVG, ParamGPRMAX, run

geometry=Geometry()

#Domaine de calcul (en cm)
# largeur
geometry.xmin=0 
geometry.xmax=40
# hauteur (elevation)
geometry.emin=0
geometry.emax = 80
# profondeur du trou en cm
geometry.dtrou = 30
# elevation du fond du trou
geometry.etrou = geometry.emax - geometry.dtrou
 # rayon du trou en cm
geometry.r=4
# hauteur d'eau imposée au fond du trou en cm
geometry.h_eau=10.0 
# pas de la maille en cm
geometry.dx = 0.1
# profondeur sous le trou (cm) jusqu'où on souhaite un maillage affiné. 
geometry.zaff= 20
#largeur horizontal de la zone affinée (cm)
geometry.waff=20
# elevation de l'affinage
geometry.eaff=geometry.etrou-geometry.zaff 
# contrainte d'angle min pour mesh 
geometry.quality=33
# maximum triangle size  (m*²)
geometry.area=5
# tupple for mesh generation 
geometry.smooth=[1,5]


#Temps d'infiltration où à lieu le calcul de chaque trace  (minutes) 
#temps=[1.00, 2.00, 3.00, 4.00, 5.00, 6.00]
temps=[0.17, 0.33, 0.50, 0.67, 0.83, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00]

# Temps max de calcul SWMS2D au delà duquel on arrète le calcul (secondes)
tmax_SWMS2D = 600
#tmax_SWMS2D = 10
   
# Definition des param gprMax
paramGPRMAX=ParamGPRMAX()
# Domaine de calcul (cm)
paramGPRMAX.xmin = geometry.xmin
paramGPRMAX.xmax = geometry.xmax
paramGPRMAX.zmin = geometry.emin
paramGPRMAX.zmax = geometry.emax
# Taille des mailles (cm)
paramGPRMAX.dx = 1.0 
# Electrical conductivity of the medium
paramGPRMAX.sigma=0.0000
# Relative dielectric permittivity of water
paramGPRMAX.eps_w=80.1
# Relative dielectric permittivity of PVC
paramGPRMAX.eps_pvc=3
# Relative dielectric permittivity of pure silice
paramGPRMAX.eps_s=2.5
# Ricker signal central frequency (Hz)
paramGPRMAX.wave_freq = 1000e6
# Frequence max du signal EM (Hz)
paramGPRMAX.freq_max = 2.8 * paramGPRMAX.wave_freq
# Distance between hole middle and source (m)
paramGPRMAX.d_emet = 0.18
# Distance between hole middle and receiving antenna (m)
paramGPRMAX.d_recept = 0.22
# param qui raffine le pas spatial (par défaut 10 d'après doc gprmax)
paramGPRMAX.spatial_step = 5
# Trace time window (ns)
paramGPRMAX.time = 30e-9
#time_step_stability_factor (pas utilisé pour le moment...)
paramGPRMAX.fac_dt = 0.2 



In [4]:
tasks=[]

for p in itertools.product(tr,ts,ti,Ks,n,alpha):
    # Definition des paramètres MVG
    paramMVG=ParamMVG()

    paramMVG.tr = p[0]
    paramMVG.ts = p[1]
    paramMVG.ti = p[2]
    paramMVG.Ks = p[3]
    paramMVG.n = p[4]
    paramMVG.alpha = p[5]
    #Porosité
    paramMVG.porosity = paramMVG.ts
    myDirName = "OUT"+repr(geometry)+"/"+repr(paramMVG)
    #tasks.append(dask.delayed(run)(geometry=geometry,paramMVG=paramMVG,paramGPRMAX=paramGPRMAX,temps=temps,tmax_SWMS2D=tmax_SWMS2D,myDirName=myDirName))
    run(geometry=geometry,paramMVG=paramMVG,paramGPRMAX=paramGPRMAX,temps=temps,tmax_SWMS2D=tmax_SWMS2D,myDirName=myDirName)
    
    



1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
tasks=dask.compute(tasks,scheduler='processes')


In [ ]:
len(tasks)

In [ ]:
dirnames=[]
for p in itertools.product(tr,ts,ti,Ks,n,alpha):
    paramMVG.tr = p[0]
    paramMVG.ts = p[1]
    paramMVG.ti = p[2]
    paramMVG.Ks = p[3]
    paramMVG.n = p[4]
    paramMVG.alpha = p[5]
    #Porosité
    paramMVG.porosity = paramMVG.ts
    dirnames.append("OUT"+repr(geometry)+"/"+repr(paramMVG))


In [ ]:
len(dirnames)

In [ ]:
len(set(dirnames))

In [ ]:
%matplotlib inline
import h5py
import math
import numpy as np
import matplotlib.pyplot as plt


# Ouverture du fichier avec le radargramme

filename1 = '/home/clemence/Albane-Porchet-GPR-mod/OUTdtrou30_rtrou4_tr10.0/ts0.36_ti0.07_tr0.03_n4.0_alpha0.01_Ks0.16000000000000003/radargram__merged.out'
f1 = h5py.File(filename1, 'r')
# Lecture des données
path = '/rxs/rx1/'
data1 = f1['%s%s' % (path, 'Ez')][:,:]

# Affichage du radargramme

#plt.figure(figsize=(20,10))
#plt.xticks(fontsize=20)
#plt.yticks(fontsize=20)
#plt.imshow(data[int(itmin0):int(itmin0+7*ifenetre),:],aspect=0.01)
#plt.imshow(data1[:,:],aspect=0.01)

filename2 = '/home/clemence/Albane-Porchet-GPR-mod/OUTdtrou30_rtrou4_tr10.0/ts0.36_ti0.07_tr0.03_n4_alpha0.01_Ks0.16000000000000003/radargram__merged.out'
f2 = h5py.File(filename2, 'r')
# Lecture des données
path = '/rxs/rx1/'
data2 = f2['%s%s' % (path, 'Ez')][:,:]

# Affichage du radargramme

#plt.figure(figsize=(10,10))
#plt.xticks(fontsize=20)
#plt.yticks(fontsize=20)
#plt.imshow(data[int(itmin0):int(itmin0+7*ifenetre),:],aspect=0.01)
plt.plot(np.arange(0,len(data2[:,11])),data2[:,11],data1[:,11])


In [ ]:
# Ouverture du fichier avec le radargramme

filename1 = '/home/clemence/Albane-Porchet-GPR-mod/OUTdtrou30_rtrou4_tr10.0.PremierEssaiDask/ts0.36_ti0.07_tr0.03_n2.25_alpha0.02_Ks0.16000000000000003/radargram__merged.out'
f1 = h5py.File(filename1, 'r')
# Lecture des données
path = '/rxs/rx1/'
data1 = f1['%s%s' % (path, 'Ez')][:,:]

# Affichage du radargramme

#plt.figure(figsize=(20,10))
#plt.xticks(fontsize=20)
#plt.yticks(fontsize=20)
#plt.imshow(data[int(itmin0):int(itmin0+7*ifenetre),:],aspect=0.01)
#plt.imshow(data1[:,:],aspect=0.01)

filename2 = '/home/clemence/Albane-Porchet-GPR-mod/OUTdtrou30_rtrou4_tr10.0.PremierEssaiDask/ts0.36_ti0.07_tr0.03_n2.25_alpha0.02_Ks0.16000000000000003.old/radargram__merged.out'

f2 = h5py.File(filename2, 'r')
# Lecture des données
path = '/rxs/rx1/'
data2 = f2['%s%s' % (path, 'Ez')][:,:]

# Affichage du radargramme

#plt.figure(figsize=(10,10))
#plt.xticks(fontsize=20)
#plt.yticks(fontsize=20)
#plt.imshow(data[int(itmin0):int(itmin0+7*ifenetre),:],aspect=0.01)
plt.plot(np.arange(0,len(data2[:,11])),data2[:,11],data1[:,11])